In [44]:
import os
os.chdir('../..')
!ls

product_impacts


python(85955) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [2]:
import pandas as pd
import numpy as np
import pickle
import csv
import time
import ast
import random
from tqdm.notebook import tqdm
from googletrans import Translator
from easynmt import EasyNMT
from langdetect import detect, LangDetectException

import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

tqdm.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# enter path here
data_dir = '../../SFS/openfoodfacts/all/'

In [4]:
chunks = 100000
data = pd.read_csv(f'{data_dir}en.openfoodfacts.org.products.csv', sep='\t', chunksize=chunks, low_memory=False, on_bad_lines='skip')

df_list = []

for chunk in data:
    df_list.append(chunk)

In [5]:
cols = ['product_name', 'categories', 'categories_tags', 'categories_en', 
        'origins', 'origins_tags', 'origins_en',
        'labels', 'labels_tags', 'labels_en',
        'stores', 'countries', 'countries_tags', 'countries_en', 
        'ingredients_text', 'ingredients_tags', 'ingredients_analysis_tags',
        'energy_100g', 'fat_100g', 'saturated-fat_100g', 
        'carbohydrates_100g', 'fiber_100g', 'proteins_100g',
        'sugars_100g', 'salt_100g', 'serving_size', 'serving_quantity']

df = pd.concat([df[cols] for df in df_list], axis=0, ignore_index=True)

In [6]:
df = df.fillna('')
df = df[df['product_name']!='']
df = df[df['product_name']!=' ']
df = df[df['countries_en']!='']

In [7]:
df['countries_en'] = df.apply(lambda row: row['countries_en'].split(','), axis=1)

In [8]:
df = df.explode('countries_en')

In [9]:
countries = df.groupby('countries_en')[['product_name']].count().reset_index()
df = df[df['countries_en'].isin(countries[countries['product_name']>1000]['countries_en'].values)]

In [10]:
df['product_name_lower'] = df['product_name'].str.lower()
df['ingredients_text_lower'] = df['ingredients_text'].str.lower()

df = df.drop_duplicates(subset=['product_name_lower', 'ingredients_text', 'countries_en'])
df = df.drop(['product_name_lower', 'ingredients_text_lower'], axis=1).reset_index(drop=True)

In [11]:
df['product_id'] = df.index
df['product_id'] = df['product_id'].apply(str)
df['product_id'] = '29112024' + df['product_id']
df['product_id'] = pd.to_numeric(df['product_id'])
first_column = df.pop('product_id') 
df.insert(0, 'product_id', first_column) 

In [12]:
def detect_language(text):
    try:
        # Only attempt detection if text has sufficient length
        if len(text.strip()) > 1:  # Adjust this threshold as needed
            return detect(text)
        else:
            return "und"  # Assign "und" (undetermined) for empty or short text
    except LangDetectException:
        return "und"  # Assign "und" for undetectable languages

In [13]:
df['ingredients_text_language'] = df.progress_apply(lambda row: detect_language(row['ingredients_text'], axis=1)
df['product_name_language'] = df.progress_apply(lambda row: detect_language(row['product_name'], axis=1)

In [14]:
df['product_name_en'] = df['product_name']
df['ingredients_text_en'] = df['ingredients_text']

In [21]:
df_eng = df[(df['ingredients_text_language']=='en') 
            | ((df['ingredients_text_language']=='und') 
               & (df['product_name_language']=='en'))].reset_index(drop=True)

df_non_eng1 = df[(df['ingredients_text_language']!='en') 
                & (df['product_name_language']!='en')
                & (df['ingredients_text']!='')].reset_index(drop=True)

df_non_eng2 = df[(df['ingredients_text_language']!='en') 
                & (df['product_name_language']!='en')
                & (df['ingredients_text']=='')].reset_index(drop=True)

In [22]:
df_eng.shape

(755225, 32)

In [23]:
df_non_eng1.shape

(556635, 32)

In [24]:
df_non_eng2.shape

(1186201, 32)

In [25]:
df_non_eng1[(df_non_eng1['product_name_language']==df_non_eng1['ingredients_text_language'])].shape

(300489, 32)

In [26]:
translation_cache = {}
translator = Translator()

def translate_with_cache(text):
    if text=='':
        return ''
    if text in translation_cache:
        return translation_cache[text]
    try:
        translated = translator.translate(text, dest='en').text
        translation_cache[text] = translated
        return translated
    except Exception as e:
        return f"Error: {e}"

In [27]:
# Translate in batches with delays
batch_size = 100
for start in range(0, len(df_non_eng1), batch_size):
    end = start + batch_size
    # print(f"Processing rows {start} to {end}")
    df_non_eng1.loc[start:end, 'ingredients_text_en'] = df_non_eng1['ingredients_text'][start:end].apply(translate_with_cache)
    # print(df_non_eng1.loc[start:end,:].sample(1)[['ingredients_text', 'ingredients_text_en']].values)
    time.sleep(5)  # Add a delay to avoid rate limiting

In [28]:
# Translate in batches with delays
batch_size = 100
for start in range(0, len(df_non_eng1), batch_size):
    end = start + batch_size
    # print(f"Processing rows {start} to {end}")
    df_non_eng1.loc[start:end, 'product_name_en'] = df_non_eng1['product_name'][start:end].apply(translate_with_cache)
    # print(df_non_eng1.loc[start:end,:].sample(1)[['product_name', 'product_name_en']].values)
    time.sleep(5)  # Add a delay to avoid rate limiting

In [29]:
# removing translation errors
df_non_eng1.loc[df_non_eng1['ingredients_text']=='na', 'ingredients_text'] = ''
df_non_eng1.loc[df_non_eng1['ingredients_text_en']=='na', 'ingredients_text_en'] = ''
df_non_eng1.loc[df_non_eng1['ingredients_text_en'].str.contains(
    'error:', case=False, regex=False), 'ingredients_text_en'] = ''
df_non_eng1.loc[df_non_eng1['product_name_en'].str.contains(
    'error:', case=False, regex=False), 'product_name_en'] = df_non_eng1[
    df_non_eng1['product_name_en'].str.contains(
    'error:', case=False, regex=False)]['product_name']

In [43]:
# Translate in batches with delays
batch_size = 100
for start in range(0, len(df_non_eng2), batch_size):
    end = start + batch_size
    # print(f"Processing rows {start} to {end}")
    df_non_eng2.loc[start:end, 'product_name_en'] = df_non_eng2['product_name'][start:end].apply(translate_with_cache)
    # print(df_non_eng2.loc[start:end,:].sample(1)[['product_name', 'product_name_en']].values)
    time.sleep(5)  # Add a delay to avoid rate limiting

In [32]:
# removing translation errors
df_non_eng2.loc[df_non_eng2['product_name_en'].str.contains(
    'error:', case=False, regex=False), 'product_name_en'] = df_non_eng2[
    df_non_eng2['product_name_en'].str.contains(
    'error:', case=False, regex=False)]['product_name']

In [39]:
df = pd.concat([df_eng, df_non_eng1, df_non_eng2], axis=0, ignore_index=True)

In [40]:
df.shape

(2498061, 32)

In [42]:
df.to_csv(f'{data_dir}openfoodfacts_lang.csv', index=False)